## **Pallier CryptoSystem**

The **Paillier Cryptosystem** is a widely used public-key encryption scheme known for its **homomorphic properties**, which allow certain types of operations to be performed on ciphertexts without decrypting them. Here's a concise summary:

1. **Inventor**: Pascal Paillier (1999).
2. **Key Features**:
   - **Additive Homomorphism**: Enables addition of plaintexts by performing operations on their ciphertexts.
   - Useful in secure voting, private data aggregation, and multiparty computation.
3. **Key Components**:
   - **Public Key**: A large modulus \( n \) (product of two large primes) and a random base \( g \).
   - **Private Key**: The prime factors of \( n \).
4. **Encryption**: 
   - Plaintext \( m \) is encrypted using a random \( r \), ensuring probabilistic encryption (different ciphertexts for the same plaintext).
   - Formula: \( c = g^m \cdot r^n \mod n^2 \).
5. **Decryption**: 
   - Uses the private key to retrieve \( m \) from \( c \).
   - Formula: \( m = \frac{L(c^{\lambda} \mod n^2)}{L(g^{\lambda} \mod n^2)} \mod n \), where \( L(x) = \frac{x-1}{n} \).
6. **Security**:
   - Based on the hardness of the **Decisional Composite Residuosity (DCR)** problem.
   - Ensures semantic security under chosen plaintext attacks.

Its homomorphic properties make it particularly suited for privacy-preserving computations in sensitive data scenarios.

### Implementation

Similar to RSA, we are firstly expected to pick two different large primes.

In [ ]:
p = 13
q = 17
assert p != q

We should perform the primeness tests for p and q.

In [ ]:
def is_prime(n):
    for i in range(2, n): # [2, n)
        if n % i == 0:
            return False
    return True

assert is_prime(p)
assert is_prime(q)

We then calculate the totient function of n where n is the multiplication of two primes we just picked.

In [ ]:
import math
n = p*q
phi = (p-1)*(q-1)
assert math.gcd(n, phi) == 1

Paillier cryptosystem depends on a L function such that L(x) = (x-1)/n. Notice that result of function L must be integer always. We will use this function in optionally key generation and decryption.

In [12]:
def lx(x):
    y = (x-1)/n
    assert y - int(y) == 0
    return int(y)

### Key Generation

We are going to generate generator g, lamdba and mu. According to the original paper,

- g should be picked randomly in range of [0, n*n]
- Lambda is the least common multiple (lcm) of p-1 and q-1. Lcm function comes with python 3.9.
- Mu should be calculated with the formula such that μ = ( L(g^λ mod n^2) )^-1

In [18]:
import random 
import math 

g = random.randint(0, n*n)
lmbda = math.lcm(p-1, q-1)
mu = pow(lx(pow(g, lmbda, n*n)), -1, n)

Instead, we are simply able to generate these values as:

- g should be equal to n plus 1
- lambda should be equal to the totient function of n or shortly phi
- mu is equal to the multiplicative inverse of lambda in mod n

In [19]:
g = n + 1
lmbda = phi * 1
mu = pow(lmbda, -1, n)

### Encryption and Decryption 

So, we are going to use following function to encrypt and decrypt messages. Notice that modulus is n squared is required in encryption whereas n itself is required in decryption.

In [21]:
def encrypt(m, r):
    assert math.gcd(r, n) == 1
    c = ( pow(g, m, n*n) * pow(r, n, n*n) ) % (n*n)
    return c
 
def decrypt(c):
    p = ( lx(pow(c, lmbda, n*n)) * mu ) % n
    return p

In [22]:
m = 123
r = random.randint(0, n)
 
c = encrypt(m, r)
p = decrypt(c)
 
assert p == m

#### Additive homomorphic implementation

Let’s encrypt a message pair with Paillier cryptosystem.

In [24]:
m1 = 123
r1 = random.randint(0, n)
 
m2 = 38
r2 = random.randint(0, n)
 
c1 = encrypt(m1, r1)
c2 = encrypt(m2, r2)

The following rule must be satisfied always because we proven Paillier is homomorphic with respect to the addition.

In [25]:
assert (c1*c2) % (n*n) == encrypt(m1 + m2, r1*r2)


In [ ]:
print (f"Cipher of {m1} is : {c1}")
print (f"Cipher of {m2} is : {c2}")
print()

print (f"Sum of {m1} and {m2} is : {m1+m2}")
print()
print(f"Sum of cipher {c1 + c2} is : {c1*c2 % (n*n)}")
print (f"Decrypted sum is : {decrypt(c1*c2 % (n*n))}")

Cipher of 123 is : 34585
Cipher of 38 is : 29862

Sum of 123 and 38 is : 161

Sum of cipher 64447 is : 34325
Decrypted sum is : 161
